<a href="https://colab.research.google.com/github/santhosh201cali/fmml2021/blob/main/Lab2_Module1_HintonHouse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>
 Module Coordinator: Thrupthi Ann John thrupthi.ann@research.iiit.ac.in <br>
 Release date: 11 October 2021 Monday <br>

 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

 ```
 This is a slightly changed version with instructions for Hinton House students
 ```

---
What needs to be done by you in this notebook:
1. Go through the code and understand whatever it wants to show.
2. The evaluation metrics or parts are divided as question sections. You may be
required to either code, write in markdown, refactor to get points in those 
sections. The intention is just that you are able to make most out of the course so don't stress over marks.
---


### Question Section 1 (Refactor)
(You are recommended to bother about this once you are done with other things)

Style guide for reference: https://google.github.io/styleguide/pyguide.html


Refactor the code and ensure the following:
- Functions have good generalized parameter names, named parameters also work.
- Imporant Functions have descriptive and concise docstring so that we are able to keep track of our functions even when the size of the code increases.



In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=0)

In [2]:
# rng = np.random.default_rng(seed=10)

In [3]:
rng.random(size=(5, 2)) * 10 +5

array([[11.36961687,  7.69786714],
       [ 5.40973524,  5.16527636],
       [13.13270239, 14.12755577],
       [11.06635776, 12.29496561],
       [10.43624991, 14.35072424]])

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. 

Let us download and examine the dataset. 

In [4]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


In [5]:
dataset.target[12]

2

Here is a function for calculating the 1-nearest neighbours

In [6]:
a = np.array([[1, 2, 3, 4, 5]])
?a.sum()

In [7]:
def NN1(x, y, query):
  """
  Find the nearest point in the dataset,
  Args:
    x : Dataset x values
    y : Class of the dataset values
    query : input to predict about

  Returns:
    label of the datapoint closest to query in the dataset
  """
  diff  = x - query  # find the difference between features. Numpy automatically takes care of the size here 
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = y[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(x, y, testdata):
  # we will run nearest neighbour for each sample in the test data 
  # and collect the predicted class in an array using list comprehension
  predlabel = np.array([NN1(x, y, i) for i in testdata])
  return predlabel


In [8]:
?NN1()

We will also define a 'random classifier', which randomly allots labels to each sample

In [9]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. 

In [10]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [11]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [12]:
testdata, testlabel, allx, ally = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(ally))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4077
Number of other samples =  16563
Percent of test data =  19.752906976744185 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [13]:
x, y, valdata, vallabel = split(allx, ally, 60/100)

What is the accuracy of our classifiers on the train dataset?

In [14]:
trainpred = NN(x, y, x)
trainAccuracy = Accuracy(y, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(x, y, x)
trainAccuracy = Accuracy(y, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.16041058669618596


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of class) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [15]:
valpred = NN(x, y, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(x, y, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.3342891638997887
Validation accuracy using random classifier is  0.17159674011469966


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. 

Now let us try another random split and check the validation accuracy

In [16]:
x, y, valdata, vallabel = split(allx, ally, 60/100)
valpred = NN(x, y, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.3332327195894959


You can run the above cell multiple times to try with different random splits. 
We notice that the accuracy is different for each run, but close together. 

Now let us compare it with the accuracy we get on the test dataset. 

In [17]:
testpred = NN(allx, ally, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.3509933774834437


### Question Set 2: Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced? 

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

#1.) How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
ans: 
validiation set will increases at 85/100 it gets accuracy by 34% (slighted decrease)
validiation set will decrease at 60/100 it gets accuracy is same  (no changes)

#2.) How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?

ans: validation set when increasing then the train data also will be affected to increase  at the time accuracy may be affected by the increasing the validation data (their is no change in both datasets accuracy) 


#3.) What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?
yes 10% is the good percentage to reserve for the validation set so that thest two factors are balanced




## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. 

In [18]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    x, y, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(x, y, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [19]:
print('Average validation accuracy is ', AverageAccuracy(allx, ally, 75/100, 10, classifier=NN))
testpred = NN(allx, ally, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.34231026752922444
test accuracy is  0.3509933774834437


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions Set 3:
#1. Does averaging the validation accuracy across multiple splits give more consistent results?

ans:) yes the validation accuracy across  multiple splits gives more consistent result. 

#2. Does it give more accurate estimate of test accuracy?

ans:) Test accuracy is depend on validation dataset.
 
#3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
ans: it increases upto one certain stage after it gets stabel .
#4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

ans :)no it cannot deal with small train dataset or validation 
